## GOBAI-O2-v1.0 (vertical mean: 0-200m; 200-600m)

In [2]:
import os
import xarray as xr
import numpy as np
import netCDF4
import seawater as sw
USER = os.environ['USER']

### GOBAI O2 Obs

In [3]:
## monthly gridded O2/T/S data; 2004-2021
a1,a2,b1,b2 = [79, 131, 149, 301]
path = f'/glade/u/home/{USER}/O2prediction/Prediction/DATA'
filename = f'{path}/GOBAI-O2-clim-v1.0.nc'
nc = netCDF4.Dataset(filename, 'r')
pres = nc.variables['pres'][:]
lat = nc.variables['lat'][a1:a2]
lon = nc.variables['lon'][b1:b2]
o2_clim = nc.variables['oxy'][:,:,a1:a2,b1:b2]
temp_clim = nc.variables['temp'][:,:,a1:a2,b1:b2]
salt_clim = nc.variables['sal'][:,:,a1:a2,b1:b2]
nc.close()
#
filename = f'{path}/GOBAI-O2-v1.0.nc'
nc = netCDF4.Dataset(filename, 'r')
o2 = nc.variables['oxy'][:,:,a1:a2,b1:b2]
temp = nc.variables['temp'][:,:,a1:a2,b1:b2]
salt = nc.variables['sal'][:,:,a1:a2,b1:b2]
nc.close()

In [4]:
# transfer dbar to m
z_r = sw.dpth(pres,lat=0.)
z_r

masked_array(data=[2.48625111579895, 9.94482421875, 19.889163970947266,
                   29.833023071289062, 39.776397705078125,
                   49.719295501708984, 59.661712646484375,
                   69.60364532470703, 79.54508972167969,
                   89.48605346679688, 99.42654418945312,
                   109.3665542602539, 119.30608367919922, 129.2451171875,
                   139.18368530273438, 149.12176513671875,
                   159.05935668945312, 168.99649047851562,
                   181.41722106933594, 198.8049774169922,
                   218.67489624023438, 238.54287719726562,
                   258.40899658203125, 278.2731628417969,
                   298.13543701171875, 317.9958190917969,
                   337.854248046875, 357.71075439453125,
                   377.56536865234375, 397.4180603027344,
                   417.26885986328125, 437.1177673339844,
                   459.4454345703125, 496.6529846191406,
                   546.2526245117188, 595

### FOSI hindcast

In [5]:
path = f'/glade/scratch/{USER}/DPLE-results'
filename = f'{path}/DPLE-FOSI_hindcast_T_S_O2_1948-2017_monthly.nc'
dset = xr.open_dataset(filename)
z_t = dset['z_t']/1e2 # m
#dz = dset['dz']/1e2 # m
z_t
#darea = dset['TAREA']/1e10 #km*km
#dvol = dz*darea

<xarray.DataArray 'z_t' (z_t: 60)>
array([5.0000000e+00, 1.5000000e+01, 2.5000000e+01, 3.5000000e+01,
       4.5000000e+01, 5.5000000e+01, 6.5000000e+01, 7.5000000e+01,
       8.5000000e+01, 9.5000000e+01, 1.0500000e+02, 1.1500000e+02,
       1.2500000e+02, 1.3500000e+02, 1.4500000e+02, 1.5500000e+02,
       1.6509840e+02, 1.7547905e+02, 1.8629128e+02, 1.9766028e+02,
       2.0971138e+02, 2.2257828e+02, 2.3640883e+02, 2.5137016e+02,
       2.6765421e+02, 2.8548364e+02, 3.0511923e+02, 3.2686798e+02,
       3.5109348e+02, 3.7822763e+02, 4.0878464e+02, 4.4337769e+02,
       4.8273672e+02, 5.2772803e+02, 5.7937286e+02, 6.3886261e+02,
       7.0756329e+02, 7.8700250e+02, 8.7882526e+02, 9.8470587e+02,
       1.1062042e+03, 1.2445669e+03, 1.4004972e+03, 1.5739464e+03,
       1.7640033e+03, 1.9689442e+03, 2.1864565e+03, 2.4139717e+03,
       2.6490012e+03, 2.8893848e+03, 3.1334048e+03, 3.3797935e+03,
       3.6276704e+03, 3.8764519e+03, 4.1257681e+03, 4.3753926e+03,
       4.6251904e+03, 4.8750835e+03, 5.1250283e+03, 5.3750000e+03],
      dtype=float32)
Coordinates:
  * z_t      (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05

### interp obs vertical grid to model vertical grid

In [6]:
o2.mask = np.isnan(o2)
np.shape(o2)

(216, 58, 52, 152)

In [7]:
o2_new = np.ma.zeros([154,35,a2-a1,b2-b1])
o2_new.mask = True
for mm in range(154):
    for ii in range(a2-a1):
        for jj in range(b2-b1):
            o2_new[mm,:,ii,jj] = np.interp(z_t[:35],z_r[:44],o2[mm,:44,ii,jj])

In [8]:
o2_new.mask = o2_new.mask + np.isnan(o2_new)

In [12]:
outfile = f'{path}/GOBAI_O2_verti_regrid_200401-201510_monthly.nc'
root = netCDF4.Dataset(outfile,'w',format='NETCDF4')
root.createDimension('nmon',154)
root.createDimension('nver',35)
root.createDimension('nlat',a2-a1)
root.createDimension('nlon',b2-b1)
wo2 = root.createVariable('O2','f4',('nmon','nver','nlat','nlon'))
wlat = root.createVariable('lat','f4',('nlat',))
wlon = root.createVariable('lon','f4',('nlon',))
wzt = root.createVariable('z_t','f4',('nver',))
wtime = root.createVariable('time','i4',('nmon',))
wo2[:,:,:,:] = o2_new[:,:,:,:]
wlat[:] = lat[:]
wlon[:] = lon[:]
wzt[:] = z_t[:35]
wtime[:] = range(154)[:]
root.close()

### Now GOBAI O2 is in 154 months x 35 vertical layers x nlat x nlon